<a href="https://colab.research.google.com/github/manisudhan/smart_traffic_violation_detection/blob/main/yolocar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit ultralytics pandas opencv-python
# Install ngrok for public tunneling
!pip install pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 89.0 MB/s eta 0:00:00


In [2]:
%%writefile main_processor.py
"""
main_processor.py

"""

import cv2
import numpy as np
import math
from ultralytics import YOLO
from datetime import datetime
import collections
import csv
import os

# Path to the example frame you uploaded (for optional debug / re-calibration)
# This file is already present in the session per your upload.
SAMPLE_FRAME_PATH = "/mnt/data/Screenshot 2025-11-20 040350.png"


def calculate_severity(speed, limit):
    if speed <= limit:
        return 0
    pct = ((speed - limit) / limit) * 100.0
    if pct < 10: return 1
    if pct < 20: return 2
    if pct < 30: return 3
    if pct < 40: return 4
    return 5


def process_video(video_path,
                  csv_path,
                  latitude,
                  longitude,
                  speed_limit_kmh,
                  is_flow_positive_y=True,
                  output_path="/content/output_speed_detection.avi",
                  debug=False):

    print("PROCESS VIDEO START")
    print(f"Video: {video_path}")
    print(f"Output: {output_path}")
    print(f"CSV: {csv_path}")
    print(f"Correct flow is positive Y? {is_flow_positive_y}")

    # ---------------------------
    # Perspective points calibrated to your uploaded frame
    # (these were manually tuned to align the road trapezoid)
    # ---------------------------
    PERSPECTIVE_POINTS = np.float32([
    [713, 126],    # Top-left
    [1283, 126],   # Top-right
    [1903, 837],   # Bottom-right
    [180, 833]     # Bottom-left
    ])

    # Target top-down rectangle size (units = virtual pixels)
    # Keep aspect ratio roughly similar to road region
    TOPDOWN_WIDTH = 800
    TOPDOWN_HEIGHT = 1200
    TARGET_POINTS = np.float32([
        [0, 0],
        [TOPDOWN_WIDTH, 0],
        [TOPDOWN_WIDTH, TOPDOWN_HEIGHT],
        [0, TOPDOWN_HEIGHT]
    ])

    # Compute homography
    M = cv2.getPerspectiveTransform(PERSPECTIVE_POINTS, TARGET_POINTS)
    Minv = cv2.getPerspectiveTransform(TARGET_POINTS, PERSPECTIVE_POINTS)

    # ---------------------------
    # SCALE: meters per unit in the top-down plane
    # This value was chosen after calibrating on your frame.
    # If you want to refine, reduce/increase slightly (0.010 - 0.015 range).
    # ---------------------------
    SCALE_FACTOR = 0.012  # meters per transformed unit (tuned for this camera)
    print(f"Using SCALE_FACTOR = {SCALE_FACTOR} m/unit")

    # ---------------------------
    # Wrong-way detection parameters
    # ---------------------------
    CORRECT_FLOW_SIGN = 1 if is_flow_positive_y else -1
    WINDOW = 15               # frames to accumulate (approx ~0.5s at 30fps)
    MIN_TOTAL_DISTANCE_M = 20 # require ~20 meters of sustained opposite movement

    # ---------------------------
    # YOLO init
    # ---------------------------
    try:
        model = YOLO("yolov8n.pt")
    except Exception as e:
        print("Warning: YOLO model load failed:", e)
        print("Ensure yolov8n.pt is available or installed inside environment.")
        raise

    # ---------------------------
    # Video IO
    # ---------------------------
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("ERROR: cannot open video:", video_path)
        return

    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps <= 0 or math.isnan(fps):
        fps = 25.0
    fps = float(fps)
    frame_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Use AVI + XVID - robust in Colab
    fourcc = cv2.VideoWriter_fourcc(*"XVID")
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_w, frame_h))

    # ---------------------------
    # Tracking and state
    # ---------------------------
    track_history = collections.defaultdict(list)       # track_id -> [(pos, t), ...]
    smoothed_speeds = collections.defaultdict(float)    # track_id -> last smoothed speed
    direction_history = collections.defaultdict(list)   # track_id -> last dy list

    violations = {}           # speed violations already logged per track
    wrong_way_violations = {} # wrong-way violations logged per track
    violation_counter = 1

    CSV_HEADER = ['Violation_ID', 'Timestamp', 'Latitude', 'Longitude',
                  'Violation_Type', 'Vehicle_Type', 'Severity']
    file_exists = os.path.isfile(csv_path)
    csv_file = open(csv_path, "a", newline="")
    csv_writer = csv.writer(csv_file)
    if not file_exists:
        csv_writer.writerow(CSV_HEADER)

    frame_idx = 0
    EMA_ALPHA = 0.3  # smoothing factor

    # ------------ optional debug: save transformed sample --------------
    if debug:
        if os.path.exists(SAMPLE_FRAME_PATH):
            sample = cv2.imread(SAMPLE_FRAME_PATH)
            if sample is not None:
                warped_sample = cv2.warpPerspective(sample, M, (TOPDOWN_WIDTH, TOPDOWN_HEIGHT))
                cv2.imwrite("/content/debug_warped_sample.png", warped_sample)
                print("Debug warped sample saved to /content/debug_warped_sample.png")
        else:
            print("Debug sample frame not found at", SAMPLE_FRAME_PATH)

    # ---------------------------
    # Frame processing loop
    # ---------------------------
    while True:
        ok, frame = cap.read()
        if not ok:
            break
        frame_idx += 1
        current_time_sec = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0

        # draw the road trapezoid for visual feedback
        cv2.polylines(frame, [PERSPECTIVE_POINTS.astype(int)], True, (0, 255, 255), 2)

        # Run tracker/detector
        try:
            results = model.track(frame, persist=True, verbose=False)
        except Exception as e:
            # If YOLO fails for a frame, skip gracefully
            print(f"YOLO track error at frame {frame_idx}: {e}")
            out.write(frame)
            continue

        # If no boxes present or ids missing, write frame and continue
        if not results or len(results) == 0 or not hasattr(results[0], "boxes") or results[0].boxes is None:
            out.write(frame)
            continue

        boxes = results[0].boxes
        # ensure ids exist
        if not hasattr(boxes, "id") or boxes.id is None:
            out.write(frame)
            continue

        xyxy = boxes.xyxy.cpu().numpy()
        ids = boxes.id.cpu().numpy().astype(int)
        cls = boxes.cls.cpu().numpy().astype(int)

        for (box, track_id, cls_id) in zip(xyxy, ids, cls):
            x1, y1, x2, y2 = box.astype(int)
            class_name = model.names[int(cls_id)]

            # only consider vehicle classes
            if class_name not in ['car', 'truck', 'bus', 'motorcycle']:
                continue

            # bottom-center point (pixel-space)
            bottom_center = (int((x1 + x2) / 2), int(y2))
            pt = np.float32([[bottom_center]]).reshape(-1, 1, 2)

            # transform to top-down plane coordinates (units)
            transformed_pt = cv2.perspectiveTransform(pt, M)
            tx, ty = float(transformed_pt[0][0][0]), float(transformed_pt[0][0][1])
            cur_pos = (tx, ty)

            # initialize history if new track
            if track_id not in track_history or len(track_history[track_id]) == 0:
                track_history[track_id].append((cur_pos, current_time_sec))
                smoothed_speeds[track_id] = 0.0
                direction_history[track_id] = []
                continue

            last_pos, last_time = track_history[track_id][-1]
            dt = current_time_sec - last_time
            if dt <= 0.0001:
                # avoid divide-by-zero or negative dt
                track_history[track_id].append((cur_pos, current_time_sec))
                continue

            dx = cur_pos[0] - last_pos[0]
            dy = cur_pos[1] - last_pos[1]

            # distance in top-down units -> convert to meters with calibrated SCALE_FACTOR
            dist_units = math.hypot(dx, dy)
            dist_meters = dist_units * SCALE_FACTOR

            # speed
            speed_mps = dist_meters / dt
            raw_speed_kmh = speed_mps * 3.6

            # EMA smoothing
            prev_speed = smoothed_speeds.get(track_id, 0.0)
            smooth_speed_kmh = EMA_ALPHA * raw_speed_kmh + (1 - EMA_ALPHA) * prev_speed
            smoothed_speeds[track_id] = smooth_speed_kmh

            # ----------------------
            # Robust Wrong-Way detection (history-based)
            # ----------------------
            direction_history[track_id].append(dy)
            if len(direction_history[track_id]) > WINDOW:
                direction_history[track_id].pop(0)
            total_y = sum(direction_history[track_id])
            is_wrong_way = (total_y * CORRECT_FLOW_SIGN) < -MIN_TOTAL_DISTANCE_M

            if is_wrong_way and track_id not in wrong_way_violations:
                wrong_way_violations[track_id] = True
                violation_id = f"W{violation_counter:04d}"
                violation_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                csv_writer.writerow([violation_id, violation_time, latitude, longitude,
                                     "Wrong Way", class_name, 5])
                print(f"[WRONG WAY] {violation_id} | TID {track_id} | {class_name}")
                violation_counter += 1

            # ----------------------
            # Speeding detection & logging
            # ----------------------
            if smooth_speed_kmh > speed_limit_kmh and track_id not in violations:
                violations[track_id] = True
                violation_id = f"V{violation_counter:04d}"
                violation_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                severity = calculate_severity(smooth_speed_kmh, speed_limit_kmh)
                csv_writer.writerow([violation_id, violation_time, latitude, longitude,
                                     "Speeding", class_name, severity])
                print(f"[SPEEDING] {violation_id} | TID {track_id} | {class_name} | {smooth_speed_kmh:.1f} km/h")
                violation_counter += 1

            # ----------------------
            # Visual overlay
            # ----------------------
            box_color = (0, 255, 0)  # green default
            if track_id in wrong_way_violations or smooth_speed_kmh > speed_limit_kmh:
                box_color = (0, 0, 255)  # red if any violation

            cv2.rectangle(frame, (x1, y1), (x2, y2), box_color, 2)
            label = f"ID {track_id}: {smooth_speed_kmh:.1f} km/h"
            if track_id in wrong_way_violations:
                label += " (WRONG)"
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, box_color, 2)

            # Save last position
            track_history[track_id].append((cur_pos, current_time_sec))

        # write output frame
        out.write(frame)

    # ---------------------------
    # Clean up
    # ---------------------------
    cap.release()
    out.release()
    csv_file.close()
    print("Processing finished.")
    print("Output saved to:", output_path)

    # If debug, also save one final debug frame snapshot
    if debug:
        try:
            # reopen output and save a thumbnail
            cap2 = cv2.VideoCapture(output_path)
            ok2, f2 = cap2.read()
            if ok2:
                cv2.imwrite("/content/debug_output_frame.png", f2)
                print("Saved /content/debug_output_frame.png")
            cap2.release()
        except Exception as e:
            print("Debug save failed:", e)

# End of file


Writing main_processor.py


In [3]:
%%writefile app.py
# This is your streamlit app script, updated for Vertical (Y-axis) Direction Input

import streamlit as st
import os
import tempfile
import pandas as pd

# Import the function from your other file
from main_processor import process_video

# --- App Configuration ---
CSV_FILE_PATH = 'violation_log.csv'
DEFAULT_LATITUDE = 13.05
DEFAULT_LONGITUDE = 80.18

# --- Streamlit UI ---
st.set_page_config(layout="wide")
st.title("🚦 Smart Traffic Violation System")
st.write("Upload a video to detect speeding and wrong-way violations.")
st.write("---")

# --- Sidebar for Configuration ---
st.sidebar.header("Camera Settings")
latitude = st.sidebar.number_input("Camera Latitude", value=DEFAULT_LATITUDE, format="%.4f")
longitude = st.sidebar.number_input("Camera Longitude", value=DEFAULT_LONGITUDE, format="%.4f")
speed_limit = st.sidebar.number_input("Speed Limit (km/h)", min_value=1.0, value=50.0)

# NEW VERTICAL INPUT: Define the expected flow direction (crucial for Wrong Way detection)
flow_direction = st.sidebar.radio(
    "Expected Traffic Flow Direction:",
    ('Top to Bottom (Positive Y)', 'Bottom to Top (Negative Y)'),
    index=0,
    help="Define the dominant movement direction in the video's perspective."
)

st.sidebar.info("Note: The perspective points are hard-coded in the script for speed calculation.")
OUTPUT_VIDEO_PATH = 'output_speed_detection.mp4'

# --- Main Page: File Upload ---
uploaded_file = st.file_uploader("Choose a video file...", type=["mp4", "mov", "avi"])

if uploaded_file is not None:
    tfile = tempfile.NamedTemporaryFile(delete=False, suffix='.mp4')
    tfile.write(uploaded_file.read())
    temp_video_path = tfile.name

    st.video(temp_video_path)

    if st.button("Process Video"):

        with st.spinner(f"Processing video with GPU... This may take a few minutes."):
            # Determine the flag based on user selection
            # Positive Y means flow is Top to Bottom (Y increases)
            is_flow_positive_y = (flow_direction == 'Top to Bottom (Positive Y)')

            # Call your main processing function with new arguments
            process_video(
                video_path=temp_video_path,
                csv_path=CSV_FILE_PATH,
                latitude=latitude,
                longitude=longitude,
                speed_limit_kmh=speed_limit,
                is_flow_positive_y=is_flow_positive_y # <-- NEW ARGUMENT
            )

        st.success("✅ Processing complete!")

        # 4. Clean up the temporary video file
        os.remove(temp_video_path)

        # --- FINAL PREVIEW: PROCESSED VIDEO ---
        st.header("Processed Video Result")
        if os.path.exists(OUTPUT_VIDEO_PATH):
            st.video(OUTPUT_VIDEO_PATH)
        else:
            st.warning("Output video file was not found.")

st.write("---")
st.header("Violation Log")

# --- Display CSV Data & Download Button ---
if os.path.exists(CSV_FILE_PATH):
    df = pd.read_csv(CSV_FILE_PATH)
    st.dataframe(df.tail(10))

    @st.cache_data
    def convert_df_to_csv(df):
        return df.to_csv(index=False).encode('utf-8')

    csv_data = convert_df_to_csv(df)

    st.download_button(
        label="Download Full Violation Log (CSV)",
        data=csv_data,
        file_name='full_violation_log.csv',
        mime='text/csv',
    )
else:
    st.info("No violations logged yet. Upload and process a video to start.")

Writing app.py


In [4]:
from pyngrok import ngrok
import subprocess
import time

# --- 1. CONFIGURATION ---
NGROK_AUTH_TOKEN = "35cQcvB6PzfwxGEyLCkS1iG2XXy_5rxt9SSbLiyoqCqL5GRc9" # <-- PASTE YOUR TOKEN HERE!
STREAMLIT_PORT = 8501

# --- 2. AUTHENTICATE ---
print("Authenticating ngrok...")
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# --- 3. START TUNNEL ---
print(f"Starting ngrok tunnel to address {STREAMLIT_PORT}...")
ngrok.kill()

# Use 'addr' for compatibility
public_url = ngrok.connect(addr=STREAMLIT_PORT)

# --- 4. START STREAMLIT & PRINT URL ---
print(f"\n✅ Ngrok Tunnel is Active. Please click the URL below:")
print(f"👉 Ngrok URL: {public_url}")

# Start Streamlit in the background
subprocess.Popen(["streamlit", "run", "app.py", "--server.port", str(STREAMLIT_PORT)])

Authenticating ngrok...
Starting ngrok tunnel to address 8501...

✅ Ngrok Tunnel is Active. Please click the URL below:
👉 Ngrok URL: NgrokTunnel: "https://unconvergent-milo-laciniate.ngrok-free.dev" -> "http://localhost:8501"


<Popen: returncode: None args: ['streamlit', 'run', 'app.py', '--server.port...>

In [ ]:
!streamlit run app.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.124.253.52:8501

  Stopping...
  Stopping...


In [ ]:
from google.colab import drive
drive.mount('/content/drive')